# Cornerstone A&D Design Data Cleaning and Analysis

This notebook provides an interactive environment for cleaning and analyzing the Cornerstone A&D Design Master Tracker data. It includes sections for:

1. Data Loading and Initial Inspection
2. Missing Value Analysis
3. Date Validation and Cleaning
4. Duplicate Detection
5. Data Type Validation
6. Value Validation
7. Data Export

## Setup and Imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Enable inline plotting
%matplotlib inline

## 1. Data Loading and Initial Inspection

In [5]:
# Load and inspect data
df = pd.read_csv('Cornerstone - A&D - Design Master Tracker.csv', low_memory=False)

In [ ]:
def load_and_inspect_data():
    """Load and display initial information about the dataset."""
    # Load the main dataset
    main_df = pd.read_csv('Cornerstone - A&D - Design Master Tracker.csv', low_memory=False)
    
    # Display basic information about the dataset
    print("Dataset Shape:", main_df.shape)
    print("\nDataset Info:")
    main_df.info()
    
    # Display first few rows
    print("\nFirst few rows:")
    print(main_df.head())
    
    return main_df

def analyze_missing_values(df):
    """Analyze and visualize missing values in the dataset."""
    # Calculate missing values
    missing_values = df.isnull().sum()
    missing_percentages = (missing_values / len(df) * 100).round(2)
    
    # Create a DataFrame with missing value information
    missing_df = pd.DataFrame({
        'Missing Values': missing_values,
        'Missing Percentage': missing_percentages
    })
    
    # Sort by missing percentage
    missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values('Missing Percentage', ascending=False)
    
    # Display missing value information
    print("Columns with missing values:")
    print(missing_df)
    
    # Create a bar plot of missing values
    plt.figure(figsize=(15, 6))
    sns.barplot(x=missing_df.index, y='Missing Percentage', data=missing_df)
    plt.xticks(rotation=45, ha='right')
    plt.title('Missing Values by Column (%)')
    plt.tight_layout()
    plt.show()

def validate_dates(df, date_col):
    """Validate dates in a specific column."""
    try:
        # Convert to datetime
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        
        # Check for future dates
        future_dates = df[df[date_col] > pd.Timestamp.now()]
        
        # Check for old dates (before 2020)
        old_dates = df[df[date_col] < pd.Timestamp('2020-01-01')]
        
        # Check for invalid dates (1900-01-01)
        invalid_dates = df[df[date_col] == pd.Timestamp('1900-01-01')]
        
        return {
            'future_dates': len(future_dates),
            'old_dates': len(old_dates),
            'invalid_dates': len(invalid_dates)
        }
    except Exception as e:
        return {'error': str(e)}

def analyze_dates(df):
    """Analyze all date columns in the dataset."""
    # Identify date columns
    date_columns = [col for col in df.columns if 'date' in col.lower() or 'issued' in col.lower() or 'approved' in col.lower()]
    
    print("Potential date columns:")
    print(date_columns)
    
    # Validate each date column
    date_validation = {}
    for col in date_columns:
        date_validation[col] = validate_dates(df, col)
    
    # Display validation results
    print("\nDate validation results:")
    for col, results in date_validation.items():
        print(f"\n{col}:")
        for key, value in results.items():
            print(f"  {key}: {value}")

def check_duplicates(df):
    """Check for duplicate rows and Site IDs."""
    # Check for duplicate rows
    duplicate_rows = df.duplicated().sum()
    print(f"Number of duplicate rows: {duplicate_rows}")
    
    if duplicate_rows > 0:
        print("\nDuplicate rows:")
        print(df[df.duplicated(keep=False)].sort_values(df.columns[0]))
    
    # Check for duplicate Site IDs
    if 'Site ID' in df.columns:
        duplicate_sites = df['Site ID'].duplicated().sum()
        print(f"\nNumber of duplicate Site IDs: {duplicate_sites}")
        
        if duplicate_sites > 0:
            print("\nDuplicate Site IDs:")
            print(df[df['Site ID'].duplicated(keep=False)].sort_values('Site ID'))

def validate_data_types(df):
    """Validate data types and check for potential issues."""
    # Display data types
    print("Data types of all columns:")
    print(pd.DataFrame({
        'Column': df.columns,
        'Data Type': df.dtypes
    }))
    
    # Check for potential type mismatches
    potential_issues = []
    
    for col in df.columns:
        # Check numeric columns
        if pd.api.types.is_numeric_dtype(df[col]):
            non_numeric = df[pd.to_numeric(df[col], errors='coerce').isna() & df[col].notna()]
            if not non_numeric.empty:
                potential_issues.append(f"Column '{col}' contains non-numeric values")
        
        # Check date columns
        if pd.api.types.is_datetime64_dtype(df[col]):
            non_date = df[pd.to_datetime(df[col], errors='coerce').isna() & df[col].notna()]
            if not non_date.empty:
                potential_issues.append(f"Column '{col}' contains non-date values")
    
    if potential_issues:
        print("\nPotential data type issues found:")
        for issue in potential_issues:
            print(f"- {issue}")
    else:
        print("\nNo data type issues found.")

def validate_values(df):
    """Validate values in key columns."""
    # Check unique values in key columns
    key_columns = ['NS Status', 'Site Type', 'Client Priority']
    for col in key_columns:
        if col in df.columns:
            print(f"\nUnique values in '{col}':")
            print(df[col].dropna().unique())
    
    # Check for inconsistent dates
    if all(col in df.columns for col in ['DD\n(F)', 'DD Issued Client (A)']):
        inconsistent_dates = df[
            (df['DD Issued Client (A)'].notna()) & 
            (df['DD\n(F)'].notna()) & 
            (df['DD Issued Client (A)'] < df['DD\n(F)'])
        ]
        
        if not inconsistent_dates.empty:
            print(f"\nFound {len(inconsistent_dates)} projects where DD was issued before the forecast date")
            print(inconsistent_dates)
        else:
            print("\nNo inconsistent dates found between DD forecast and actual dates")

def clean_and_export_data(df):
    """Clean the dataset and export it to CSV."""
    # Create a cleaned version of the dataset
    cleaned_df = df.copy()
    
    # Add your cleaning steps here
    # For example:
    # - Convert dates to proper format
    # - Handle missing values
    # - Remove duplicates
    # - Fix data type issues
    
    # Export the cleaned dataset
    cleaned_df.to_csv('cleaned_cornerstone_data.csv', index=False)
    print("Cleaned dataset exported to 'cleaned_cornerstone_data.csv'")

def main():
    """Main function to run all data cleaning and analysis steps."""
    # Load and inspect data
    print("Loading and inspecting data...")
    df = load_and_inspect_data()
    
    # Analyze missing values
    print("\nAnalyzing missing values...")
    analyze_missing_values(df)
    
    # Analyze dates
    print("\nAnalyzing dates...")
    analyze_dates(df)
    
    # Check duplicates
    print("\nChecking for duplicates...")
    check_duplicates(df)
    
    # Validate data types
    print("\nValidating data types...")
    validate_data_types(df)
    
    # Validate values
    print("\nValidating values...")
    validate_values(df)
    
    # Clean and export data
    print("\nCleaning and exporting data...")
    clean_and_export_data(df)

if __name__ == "__main__":
    main() 